In [1]:
import os
import json
import re
from collections import defaultdict
import math
import pypdf
import spacy

# --- 1. Paramètres de Configuration et Préparation spaCy ---

# Chemins relatifs à Plant_search/plant-search-engine/indexer/
DOSSIER_PLANTES_JSON = "../docs/Plantes"
DOSSIER_CONCEPTS_PDF = "../docs/Concepts"

# FICHIER D'ENTRÉE POUR LE STOCK SCIENTIFIQUE
FICHIER_STOCK_SCIENTIFIQUE = "../docs/mot_scientifique/protected_terms.json" 

# Fichiers de sortie (dans le dossier indexer/)
FICHIER_INDEX_SORTIE = "Base_Index.json"
FICHIER_LONGUEURS_SORTIE = "document_lengths.json"

LANGUE = 'fr_core_news_sm'
MAX_NGRAM_SCIENTIFIQUE = 4 # Longueur maximale des N-grammes à vérifier

# Chargement du modèle spaCy
try:
    nlp = spacy.load(LANGUE)
except OSError:
    print(f"\n🚨 Erreur: Modèle spaCy '{LANGUE}' non trouvé.")
    print(f"Veuillez l'installer avec : python -m spacy download {LANGUE}")
    exit()

# Stock des termes scientifiques chargé au démarrage
TERMES_SCIENTIFIQUES_STOCK = set()

def charger_stock_scientifique(filepath: str) -> set:
    """
    Charge le stock scientifique depuis un fichier JSON. 
    Les termes sont convertis en minuscules pour la comparaison.
    """
    stock = set()
    if not os.path.exists(filepath):
        print(f"⚠️ Avertissement: Fichier de stock scientifique '{filepath}' non trouvé. Le stock est vide.")
        return stock
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
            if isinstance(data, list):
                stock.update({item.lower().strip() for item in data})
            elif isinstance(data, dict):
                stock.update({key.lower().strip() for key in data.keys()})
            else:
                print(f"⚠️ Avertissement: Format du fichier '{filepath}' non reconnu.")

    except Exception as e:
        print(f"❌ Erreur lors du chargement du stock scientifique: {e}")
        return set()
    
    print(f"✅ Stock de {len(stock)} termes scientifiques/spéciaux chargés.")
    return stock

# Charger le stock dès que possible
TERMES_SCIENTIFIQUES_STOCK = charger_stock_scientifique(FICHIER_STOCK_SCIENTIFIQUE)


# --- 2. Fonctions d'Extraction et de Traitement du Texte ---

def extraire_texte_json_avec_termes_separes(filepath: str) -> tuple[str, list[str], list[str]]:
    """
    Extrait le texte général, les termes numériques, et les termes Darija.
    Les termes Darija sont extraits directement (tels quels, en minuscule).
    """
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Erreur d'extraction/lecture JSON à {filepath}: {e}")
        return "", [], []
        
    texte_general = []
    termes_nombres = []
    termes_darija_directs = [] # Pour les termes Darija (pris tels quels)
    
    def parcourir_dict(d):
        for key, value in d.items():
            if key in ('url', 'urls', 'galerie_images', 'id'):
                continue
                
            # GESTION SPÉCIALE DES NOMS DARIJA (Indexation Directe)
            if key == 'noms_darija' and isinstance(value, list):
                # Ajout des termes Darija en minuscules, tels quels, sans lemmatisation
                termes_darija_directs.extend([t.lower().strip() for t in value if t])
                continue # NE PAS AJOUTER AU TEXTE GÉNÉRAL

            # Le reste (y compris 'nom_scientifique') est ajouté au texte général
            if isinstance(value, dict):
                parcourir_dict(value)
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, dict):
                        parcourir_dict(item)
                    elif isinstance(item, str):
                        texte_general.append(item)
            elif isinstance(value, str):
                texte_general.append(value)
    
    parcourir_dict(data)
    
    texte_complet = ' '.join(texte_general)
    
    # 3. Extraction des Nombres (sans leurs unités)
    nombres = re.findall(r'(\d+[\.,]\d+|\d+)\s*(%|ml|g|cm|mm|m|l)?', texte_complet, re.IGNORECASE)
    
    for nombre, unite in nombres:
        terme_nombre = nombre.replace(',', '.') 
        termes_nombres.append(terme_nombre)
    
    # Nettoyer le texte général des chiffres pour le traitement NLP
    texte_nettoye = re.sub(r'(\d+[\.,]\d+|\d+)\s*(%|ml|g|cm|mm|m|l)?', ' ', texte_complet, flags=re.IGNORECASE)
        
    return texte_nettoye, termes_nombres, termes_darija_directs

def extraire_texte_pdf(filepath: str) -> str:
    """Extrait le texte d'un fichier PDF."""
    texte = ""
    try:
        with open(filepath, 'rb') as f:
            reader = pypdf.PdfReader(f)
            for page in reader.pages:
                texte += page.extract_text() + "\n"
    except Exception as e:
        print(f"⚠️ Erreur lors de l'extraction de texte du PDF {filepath}. Erreur: {e}")
        return ""
    return texte

def identifier_termes_scientifiques(tokens: list, stock_scientifique: set, max_n: int) -> tuple[list, list]:
    """
    Identifie et extrait les termes scientifiques/spéciaux multi-mots (N-grammes)
    en comparant le texte tokenisé au stock.
    """
    mots_scientifiques = []
    indices_captures = set() 
    
    i = 0
    while i < len(tokens):
        if i in indices_captures:
            i += 1
            continue
        
        terme_trouve = False
        
        # Teste les N-grammes de la plus grande taille à la plus petite (ex: 4, 3, 2, 1)
        for n in range(min(max_n, len(tokens) - i), 0, -1):
            
            ngram_tokens = tokens[i:i+n]
            
            # Concaténation des tokens pour la comparaison
            ngram_candidat = " ".join(ngram_tokens)
            ngram_candidat_lower = ngram_candidat.lower().strip()
            
            if ngram_candidat_lower in stock_scientifique:
                # Terme scientifique/spécial trouvé ! On l'ajoute tel quel.
                mots_scientifiques.append(ngram_candidat_lower)
                
                # Marquer tous les tokens de ce N-gramme comme capturés
                for j in range(n):
                    indices_captures.add(i + j)
                
                terme_trouve = True
                i += n 
                break
        
        if not terme_trouve:
            i += 1
            
    # Extraction des tokens restants (mots généraux)
    tokens_restants = [tokens[i] for i in range(len(tokens)) if i not in indices_captures]
    
    return mots_scientifiques, tokens_restants


def normaliser_texte_lemmatisation_filtree(tokens_restants: list) -> list[str]:
    """
    Lemmatisation stricte des tokens restants (mots généraux). 
    Applique le filtrage : minuscules, enlève ponctuation, stop words, symboles, puis lemmatise.
    """
    texte_restant = " ".join(tokens_restants) 
    doc = nlp(texte_restant.lower())
    stop_words = nlp.Defaults.stop_words
    mots_normalises = []
    
    for token in doc:
        lemma_text = token.lemma_.lower().strip()
        
        # Application du filtrage strict pour les mots généraux
        if (
            not token.is_punct and      # Enlève la ponctuation
            token.is_alpha and          # Enlève les symboles et caractères non alphabétiques
            not token.is_stop and       # Enlève les stop words
            len(lemma_text) > 1         # Longueur minimale du lemme
        ):
            mots_normalises.append(lemma_text) 
            
    return mots_normalises

# --- 3. Classe Principale d'Indexation ---

class IndexeurBotanique:
    
    def __init__(self, json_dir, pdf_dir):
        self.json_dir = json_dir
        self.pdf_dir = pdf_dir
        # Structure TF simple: { 'mot': { 'document_id': fréquence } }
        self.index = defaultdict(lambda: defaultdict(int))
        self.doc_lengths = {}
        
    def _creer_doc_id(self, filename: str, doc_type: str) -> str:
        """Crée l'ID du document en utilisant LE NOM DE FICHIER COMPLET (avec extension)."""
        return filename

    def parcourir_documents(self):
        """Parcourt et indexe TOUS les JSON et TOUS les PDF."""
        documents_a_indexer = []
        
        print(f"Recherche des fichiers JSON dans '{self.json_dir}'...")
        if os.path.isdir(self.json_dir):
            for filename in os.listdir(self.json_dir):
                if filename.endswith(".json"):
                    doc_id = self._creer_doc_id(filename, 'json')
                    documents_a_indexer.append({'id': doc_id, 'type': 'json', 'path': os.path.join(self.json_dir, filename)})
        else:
            print(f"❌ Erreur: Dossier JSON '{self.json_dir}' introuvable.")


        print(f"Recherche des fichiers PDF dans '{self.pdf_dir}'...")
        if os.path.isdir(self.pdf_dir):
            for filename in os.listdir(self.pdf_dir):
                if filename.endswith(".pdf"):
                    doc_id = self._creer_doc_id(filename, 'pdf')
                    documents_a_indexer.append({'id': doc_id, 'type': 'pdf', 'path': os.path.join(self.pdf_dir, filename)})
        else:
            print(f"❌ Erreur: Dossier PDF '{self.pdf_dir}' introuvable.")
                
        print(f"\nDébut de l'indexation de {len(documents_a_indexer)} documents...")
        
        for doc_info in documents_a_indexer:
             self.indexer_document(doc_info)
        print("\n--- Indexation Terminée ---")

    # La fonction parcourir_documents_test() est conservée mais non utilisée dans le main
    def parcourir_documents_test(self, nom_json: str, nom_pdf: str):
         """Parcourt et indexe seulement 1 JSON et 1 PDF spécifiques pour le test."""
         # ... (votre implémentation)

    def indexer_document(self, doc_info):
        """Extrait, normalise et construit l'index TF pour un document unique."""
        doc_id = doc_info['id']
        filepath = doc_info['path']
        
        mots_indexables = []
        texte_brut_pour_tokenisation = ""
        termes_nombres = []
        termes_darija_directs = []

        # 1. Extraction du texte, nombres et termes Darija
        if doc_info['type'] == 'json':
            texte_brut_pour_tokenisation, termes_nombres, termes_darija_directs = extraire_texte_json_avec_termes_separes(filepath)
            
        elif doc_info['type'] == 'pdf':
            texte_complet_pdf = extraire_texte_pdf(filepath)
            if not texte_complet_pdf: return 
            
            # Extraction des Nombres (PDF)
            nombres = re.findall(r'(\d+[\.,]\d+|\d+)\s*(%|ml|g|cm|mm|m|l)?', texte_complet_pdf, re.IGNORECASE)
            for nombre, unite in nombres:
                 termes_nombres.append(nombre.replace(',', '.')) 
            
            # Nettoyage du texte général des chiffres pour le traitement NLP
            texte_brut_pour_tokenisation = re.sub(r'(\d+[\.,]\d+|\d+)\s*(%|ml|g|cm|mm|m|l)?', ' ', texte_complet_pdf, flags=re.IGNORECASE)

        # 2. AJOUT DIRECT des Termes (Nombres et Darija)
        mots_indexables.extend(termes_nombres)
        mots_indexables.extend(termes_darija_directs) 
        
        # 3. Tokenisation pour la reconnaissance des N-grammes
        doc = nlp(texte_brut_pour_tokenisation)
        tokens_bruts = [token.text for token in doc if not token.is_space]
        
        # 4. Reconnaissance des Termes Scientifiques (non lemmatisés)
        mots_scientifiques, tokens_restants = identifier_termes_scientifiques(
            tokens_bruts, TERMES_SCIENTIFIQUES_STOCK, MAX_NGRAM_SCIENTIFIQUE
        )
        mots_indexables.extend(mots_scientifiques) 
        
        # 5. Lemmatisation des Mots Généraux Restants
        mots_lemmatises = normaliser_texte_lemmatisation_filtree(tokens_restants)
        mots_indexables.extend(mots_lemmatises)

        if not mots_indexables: return

        # 6. Enregistrement de la longueur et construction de l'index
        self.doc_lengths[doc_id] = len(mots_indexables)
        
        for mot in mots_indexables:
            if mot:
                self.index[mot][doc_id] += 1 

    def enregistrer_longueurs(self, output_file):
        """Enregistre les longueurs des documents."""
        os.makedirs(os.path.dirname(output_file) or '.', exist_ok=True)
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(self.doc_lengths, f, ensure_ascii=False, indent=2)
        print(f"✅ Longueurs des documents (Token Counts) enregistrées dans '{output_file}'.")

    def enregistrer_index(self, output_file):
        """Enregistre l'index inversé final (Base d'Index) au format simplifié."""
        os.makedirs(os.path.dirname(output_file) or '.', exist_ok=True)
        
        index_a_sauvegarder = dict(self.index)
        
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(index_a_sauvegarder, f, ensure_ascii=False, indent=2)
            
        print(f"✅ Base d'Index finale ('{output_file}') enregistrée.")


# --- 4. Exécution du Script Principal (Mode Production) ---

if __name__ == "__main__":
    
    print("--- Démarrage de l'Indexation Complète (Méthode Avancée) ---")

    # Le stock scientifique est déjà chargé au début du script

    # 1. Initialisation de l'Indexeur
    indexeur = IndexeurBotanique(
        json_dir=DOSSIER_PLANTES_JSON,
        pdf_dir=DOSSIER_CONCEPTS_PDF
    )
    
    # 2. Exécution de l'indexation sur TOUS les fichiers
    indexeur.parcourir_documents()
    
    # 3. Enregistrement des Fichiers de Sortie
    indexeur.enregistrer_longueurs(FICHIER_LONGUEURS_SORTIE)
    indexeur.enregistrer_index(FICHIER_INDEX_SORTIE)

    print("\n--- Processus d'Indexation Complet Terminé ---")
    print(f"L'index final ('{FICHIER_INDEX_SORTIE}') et les longueurs ('{FICHIER_LONGUEURS_SORTIE}') sont générés dans le dossier de l'indexeur.")

✅ Stock de 615 termes scientifiques/spéciaux chargés.
--- Démarrage de l'Indexation Complète (Méthode Avancée) ---
Recherche des fichiers JSON dans '../docs/Plantes'...
Recherche des fichiers PDF dans '../docs/Concepts'...

Début de l'indexation de 95 documents...

--- Indexation Terminée ---
✅ Longueurs des documents (Token Counts) enregistrées dans 'document_lengths.json'.
✅ Base d'Index finale ('Base_Index.json') enregistrée.

--- Processus d'Indexation Complet Terminé ---
L'index final ('Base_Index.json') et les longueurs ('document_lengths.json') sont générés dans le dossier de l'indexeur.
